---
title: "Week 3: Working with Geospatial Foundation Models"
subtitle: "Hands-on practice with TerraTorch and pretrained models"
jupyter: geoai
format:
  html:
    toc: true
    toc-depth: 3
    code-fold: false
---





## Introduction

This week we'll work with pretrained geospatial foundation models (GFMs) using TerraTorch, a framework that simplifies fine-tuning and inference with state-of-the-art models. You'll learn to use models like Prithvi and Clay for practical remote sensing tasks.

:::{.callout-tip}
## Learning Goals
By the end of this session, you will:

- Understand the architecture of geospatial foundation models
- Load and use pretrained models for feature extraction
- Fine-tune models on custom datasets
- Compare different GFM architectures
- Apply models to real satellite imagery
:::

## Session Overview

Today's workflow focuses on practical model usage:

| Step | Activity | Tools | Output |
|------|----------|-------|--------|
| 1 | Model architecture exploration | TerraTorch | Understanding |
| 2 | Feature extraction | PyTorch, TerraTorch | Embeddings |
| 3 | Simple classification task | TerraTorch, TorchGeo | Trained model |
| 4 | Model comparison | Multiple backbones | Performance metrics |
| 5 | Real-world inference | Satellite imagery | Predictions |

---

## Step 1: Environment Setup and Model Loading

Let's start by setting up our environment and understanding the available models.

### Import Libraries and Check Setup


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

### Model Utilities → `geogfm/utils/terratorch_utils.py`

Let's create reusable utilities for working with TerraTorch models.


In [ ]:
"""Utilities for working with TerraTorch models."""

import torch
import torch.nn as nn
from typing import Dict, Optional, Tuple, List
import numpy as np


def load_pretrained_model(
    model_name: str,
    num_classes: Optional[int] = None,
    task: str = "classification",
    device: str = "cpu"
) -> nn.Module:
    """
    Load a pretrained model.

    Args:
        model_name: Name of the model (e.g., 'prithvi_100m', 'clay_v1')
        num_classes: Number of output classes (for classification/segmentation)
        task: Task type ('classification', 'segmentation', 'embedding')
        device: Device to load model on

    Returns:
        Loaded model

    Note:
        This function requires terratorch to be installed.
        For this demo, we use simplified models instead.
    """
    try:
        from terratorch.models import get_model

        model_config = {
            'backbone': model_name,
            'task': task,
        }

        if num_classes is not None:
            model_config['num_classes'] = num_classes

        model = get_model(**model_config)
        model = model.to(device)
        model.eval()

        return model
    except ImportError:
        raise ImportError(
            "terratorch is required for loading foundation models. "
            "Install with: pip install terratorch"
        )


def extract_features(
    model: nn.Module,
    images: torch.Tensor,
    layer: str = "last",
    device: str = "cpu"
) -> torch.Tensor:
    """
    Extract features from a model.

    Args:
        model: PyTorch model
        images: Input images (B, C, H, W)
        layer: Which layer to extract from
        device: Device to use

    Returns:
        Feature tensor
    """
    model.eval()
    images = images.to(device)

    with torch.no_grad():
        if hasattr(model, 'encode'):
            features = model.encode(images)
        else:
            features = model(images)

    return features


def get_model_info(model: nn.Module) -> Dict:
    """
    Get information about a model.

    Args:
        model: PyTorch model

    Returns:
        Dictionary with model info
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    return {
        'total_parameters': total_params,
        'trainable_parameters': trainable_params,
        'frozen_parameters': total_params - trainable_params,
        'model_type': type(model).__name__
    }


def prepare_satellite_image(
    image_path: str,
    target_bands: Optional[List[int]] = None,
    normalize: bool = True
) -> torch.Tensor:
    """
    Load and prepare a satellite image for model input.

    Args:
        image_path: Path to image file
        target_bands: Band indices to use (None = all)
        normalize: Whether to normalize to [0, 1]

    Returns:
        Tensor of shape (1, C, H, W)
    """
    try:
        import rasterio
    except ImportError:
        raise ImportError("rasterio required for image loading. Install with: pip install rasterio")

    with rasterio.open(image_path) as src:
        if target_bands is None:
            data = src.read()
        else:
            data = src.read(target_bands)

    # Convert to float and normalize
    data = data.astype(np.float32)

    if normalize:
        # Handle NaN values
        valid_mask = ~np.isnan(data)
        if valid_mask.any():
            data_min = np.nanmin(data)
            data_max = np.nanmax(data)
            data = (data - data_min) / (data_max - data_min + 1e-8)
            data = np.nan_to_num(data, nan=0.0)

    # Add batch dimension and convert to tensor
    tensor = torch.from_numpy(data).unsqueeze(0)

    return tensor

### Model Comparison Utilities → `geogfm/utils/terratorch_utils.py`


In [ ]:
def compare_models(
    model_names: List[str],
    sample_input: torch.Tensor,
    device: str = "cpu"
) -> pd.DataFrame:
    """
    Compare multiple models on the same input.

    Args:
        model_names: List of model names to compare
        sample_input: Sample input tensor
        device: Device to use

    Returns:
        DataFrame with comparison results
    """
    results = []

    for model_name in model_names:
        try:
            model = load_pretrained_model(
                model_name=model_name,
                task="embedding",
                device=device
            )

            info = get_model_info(model)

            # Measure inference time
            import time
            start = time.time()
            with torch.no_grad():
                output = extract_features(model, sample_input, device=device)
            elapsed = time.time() - start

            results.append({
                'model': model_name,
                'total_params': info['total_parameters'],
                'trainable_params': info['trainable_parameters'],
                'inference_time_ms': elapsed * 1000,
                'output_shape': str(tuple(output.shape))
            })

        except Exception as e:
            print(f"Error loading {model_name}: {e}")
            continue

    return pd.DataFrame(results)


def visualize_embeddings(
    embeddings: np.ndarray,
    labels: Optional[np.ndarray] = None,
    method: str = "pca",
    n_components: int = 2
) -> plt.Figure:
    """
    Visualize high-dimensional embeddings in 2D.

    Args:
        embeddings: Feature embeddings (N, D)
        labels: Optional labels for coloring
        method: Dimensionality reduction method ('pca', 'tsne')
        n_components: Number of components

    Returns:
        Matplotlib figure
    """
    from sklearn.decomposition import PCA
    from sklearn.manifold import TSNE

    if method == "pca":
        reducer = PCA(n_components=n_components)
    elif method == "tsne":
        reducer = TSNE(n_components=n_components, random_state=42)
    else:
        raise ValueError(f"Unknown method: {method}")

    reduced = reducer.fit_transform(embeddings)

    fig, ax = plt.subplots(figsize=(10, 8))

    if labels is not None:
        scatter = ax.scatter(
            reduced[:, 0], reduced[:, 1],
            c=labels, cmap='tab10', alpha=0.6, s=50
        )
        plt.colorbar(scatter, ax=ax, label='Class')
    else:
        ax.scatter(
            reduced[:, 0], reduced[:, 1],
            alpha=0.6, s=50
        )

    ax.set_xlabel(f'{method.upper()} Component 1')
    ax.set_ylabel(f'{method.upper()} Component 2')
    ax.set_title(f'Embedding Visualization ({method.upper()})')
    ax.grid(True, alpha=0.3)

    return fig

---

## Step 2: Working with Pretrained Models

Now let's load and explore a simple pretrained model.

### Demo: Load and Inspect a Model


In [ ]:
# Helper function for model info
def get_model_info(model):
    """Get information about a model."""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {
        'total_parameters': total_params,
        'trainable_parameters': trainable_params,
        'frozen_parameters': total_params - trainable_params,
        'model_type': type(model).__name__
    }

# Create a simple demonstration model
# This represents a feature extractor similar to what foundation models provide
class SimpleFeatureExtractor(nn.Module):
    """
    Lightweight feature extractor demonstrating foundation model concepts.

    In practice, you would load pretrained models like Prithvi or Clay.
    This simplified version shows the same workflow without dependencies.
    """
    def __init__(self, in_channels=4, embed_dim=256):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, embed_dim, kernel_size=3, padding=1),
            nn.AdaptiveAvgPool2d(1)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x.squeeze(-1).squeeze(-1)

model = SimpleFeatureExtractor(in_channels=4).to(device)  # 4 bands: RGB+NIR
model_info = get_model_info(model)

print("Feature Extractor Model Created")
print(f"Total parameters: {model_info['total_parameters']:,}")
print(f"Trainable parameters: {model_info['trainable_parameters']:,}")
print(f"Model type: {model_info['model_type']}")

print("\nThis model demonstrates the same workflow as full foundation models:")
print("- Feature extraction from satellite imagery")
print("- Transfer learning for downstream tasks")
print("- Fine-tuning strategies")

### Load Real Satellite Data

Let's use actual Sentinel-2 imagery instead of synthetic data. We'll use our `geogfm` library to efficiently download and process a small area.

:::{.callout-note}
## Download Optimization Strategy
To keep downloads fast and manageable:

1. **Small AOI**: ~5km x 5km area around Santa Barbara
2. **Low cloud cover**: < 5% ensures clean data
3. **20m resolution**: Faster than 10m, still good quality
4. **4 bands only**: RGB + NIR (B02, B03, B04, B08)
5. **Cloud masking**: Filters out contaminated pixels
6. **Patch extraction**: Small 64x64 pixel patches

Total download: ~50-100 MB instead of several GB!
:::


In [ ]:
# Import geogfm functions for data loading
from geogfm.c01 import (
    setup_planetary_computer_auth,
    search_sentinel2_scenes,
    load_sentinel2_bands,
    get_subset_from_scene
)
from geogfm.c02 import load_scene_with_cloudmask

# Setup authentication
setup_planetary_computer_auth()

# Define a small AOI in Santa Barbara (coastal area with diverse land cover)
# This small area (~5km x 5km) downloads quickly but has good variety
santa_barbara_bbox = [-119.85, 34.40, -119.75, 34.48]

print("Searching for high-quality Sentinel-2 scenes...")
print(f"AOI: Santa Barbara coastal area")

# Search for clear scenes (low cloud cover)
scenes = search_sentinel2_scenes(
    bbox=santa_barbara_bbox,
    date_range="2023-06-01/2023-08-31",  # Summer for clear weather
    cloud_cover_max=5,  # Very low cloud cover
    limit=5
)

print(f"\nFound {len(scenes)} clear scenes")
if scenes:
    best_scene = scenes[0]
    print(f"Using scene: {best_scene.id}")
    print(f"Date: {best_scene.properties['datetime'][:10]}")
    print(f"Cloud cover: {best_scene.properties['eo:cloud_cover']:.1f}%")

### Extract Patches from Real Scene

Now let's create a function to extract small patches from the satellite scene. This approach keeps memory usage low while providing real spectral data.


In [ ]:
def extract_patches_from_scene(
    item,
    subset_bbox,
    patch_size=64,
    stride=32,
    bands=['B02', 'B03', 'B04', 'B08'],
    target_patches=200
):
    """
    Extract fixed-size patches from a satellite scene.

    Args:
        item: STAC item
        subset_bbox: Spatial subset to load
        patch_size: Size of each patch in pixels
        stride: Stride between patches (smaller = more overlap)
        bands: Bands to load
        target_patches: Approximate number of patches to extract

    Returns:
        patches: Tensor of shape (N, C, H, W)
        valid_mask: Boolean mask indicating valid (non-cloudy) patches
    """
    # Load scene data with cloud masking
    print("Loading scene data...")
    scene_data, valid_fraction = load_scene_with_cloudmask(
        item,
        target_crs='EPSG:32610',  # UTM Zone 10N for Santa Barbara
        target_resolution=20,  # 20m resolution for faster processing
        good_pixel_classes=[4, 5, 6],  # Vegetation, bare soil, water
        subset_bbox=subset_bbox
    )

    if scene_data is None or valid_fraction < 0.3:
        raise ValueError(f"Insufficient valid data (fraction: {valid_fraction:.2f})")

    print(f"Valid pixel fraction: {valid_fraction:.1%}")

    # Get band data and cloud mask
    band_arrays = []
    for band_name in ['red', 'green', 'blue', 'nir']:
        if band_name in scene_data:
            band_data = scene_data[band_name].values
            band_arrays.append(band_data)

    cloud_mask = scene_data['cloud_mask'].values

    # Stack bands: (C, H, W)
    scene_array = np.stack(band_arrays, axis=0)
    print(f"Scene array shape: {scene_array.shape}")

    # Extract patches
    patches = []
    valid_masks = []

    C, H, W = scene_array.shape

    # Calculate patch positions
    patch_positions = []
    for y in range(0, H - patch_size + 1, stride):
        for x in range(0, W - patch_size + 1, stride):
            patch_positions.append((y, x))

    # Randomly sample if we have too many
    if len(patch_positions) > target_patches:
        indices = np.random.choice(len(patch_positions), target_patches, replace=False)
        patch_positions = [patch_positions[i] for i in indices]

    print(f"Extracting {len(patch_positions)} patches...")

    for y, x in patch_positions:
        # Extract patch
        patch = scene_array[:, y:y+patch_size, x:x+patch_size]
        mask_patch = cloud_mask[y:y+patch_size, x:x+patch_size]

        # Check if patch is mostly valid
        valid_fraction = np.mean(mask_patch)

        if valid_fraction > 0.8:  # At least 80% valid pixels
            # Normalize patch to [0, 1] using percentile scaling
            patch_normalized = np.zeros_like(patch)
            for c in range(patch.shape[0]):
                band = patch[c]
                valid_data = band[~np.isnan(band)]
                if len(valid_data) > 0:
                    p2, p98 = np.percentile(valid_data, [2, 98])
                    band_norm = (band - p2) / (p98 - p2 + 1e-8)
                    band_norm = np.clip(band_norm, 0, 1)
                    band_norm = np.nan_to_num(band_norm, 0)
                    patch_normalized[c] = band_norm

            patches.append(patch_normalized)
            valid_masks.append(True)
        else:
            valid_masks.append(False)

    # Convert to torch tensors
    if patches:
        patches_tensor = torch.from_numpy(np.stack(patches)).float()
        print(f"Extracted {len(patches)} valid patches of shape {patches_tensor[0].shape}")
        return patches_tensor, np.array(valid_masks)
    else:
        raise ValueError("No valid patches extracted")

# Extract patches from the scene
print("\nExtracting patches from satellite scene...")
sample_patches, valid_mask = extract_patches_from_scene(
    best_scene,
    subset_bbox=santa_barbara_bbox,
    patch_size=64,
    stride=48,  # Some overlap for diversity
    target_patches=200
)

print(f"\nPatch dataset created:")
print(f"Number of patches: {len(sample_patches)}")
print(f"Patch shape: {sample_patches[0].shape}")
print(f"Data type: {sample_patches.dtype}")

### Create Pseudo-Labels for Classification

Since we're using real data without ground truth labels, we'll create pseudo-labels based on spectral characteristics. This demonstrates how foundation models can learn from spectral patterns.


In [ ]:
def create_spectral_labels(patches):
    """
    Create pseudo-labels based on spectral characteristics.

    Uses NDVI and other indices to classify into approximate land cover types.
    This is for demonstration - real applications would use ground truth.
    """
    labels = []

    for patch in patches:
        # Calculate spectral indices
        red = patch[0].numpy()  # B02
        green = patch[1].numpy()  # B03
        blue = patch[2].numpy()  # B04
        nir = patch[3].numpy()  # B08

        # NDVI
        ndvi = (nir - red) / (nir + red + 1e-8)

        # Mean values
        ndvi_mean = np.mean(ndvi)
        nir_mean = np.mean(nir)
        red_mean = np.mean(red)
        blue_mean = np.mean(blue)

        # Simple classification based on spectral signatures
        if blue_mean > 0.6 and ndvi_mean < 0:
            label = 0  # Water (high blue, low NDVI)
        elif ndvi_mean > 0.5:
            label = 1  # Dense vegetation (high NDVI)
        elif nir_mean < 0.3 and red_mean > 0.4:
            label = 2  # Urban/built-up (low NIR, moderate red)
        elif ndvi_mean < 0.2 and red_mean > 0.5:
            label = 3  # Bare soil (low NDVI, high red)
        else:
            label = 4  # Mixed/other (everything else)

        labels.append(label)

    return torch.tensor(labels)

# Create labels
sample_labels = create_spectral_labels(sample_patches)

print(f"Created spectral-based labels")
print(f"Label distribution:")
class_names = ['Water', 'Vegetation', 'Urban', 'Bare Soil', 'Mixed']
for i, name in enumerate(class_names):
    count = (sample_labels == i).sum().item()
    print(f"  {name}: {count} patches ({count/len(sample_labels)*100:.1f}%)")

### Visualize Real Satellite Patches

Let's look at some of the patches we extracted from the satellite scene.


In [ ]:
# Visualize a sample of patches with their labels
fig, axes = plt.subplots(3, 6, figsize=(18, 9))
axes = axes.ravel()

# Sample patches from each class
for i, class_idx in enumerate(range(5)):
    # Get patches of this class
    class_mask = sample_labels == class_idx
    class_indices = torch.where(class_mask)[0]

    # Show up to 3 examples per class
    n_show = min(3, len(class_indices))
    for j in range(n_show):
        idx = class_indices[j].item()
        ax = axes[i * 3 + j]

        # Create false color composite (NIR-R-G)
        nir = sample_patches[idx, 3].numpy()
        red = sample_patches[idx, 0].numpy()
        green = sample_patches[idx, 1].numpy()

        img = np.stack([nir, red, green], axis=2)
        img = np.clip(img, 0, 1)

        ax.imshow(img)
        ax.axis('off')
        ax.set_title(f"{class_names[class_idx]}", fontsize=10, fontweight='bold')

    # Hide unused axes
    for j in range(n_show, 3):
        axes[i * 3 + j].axis('off')

# Hide remaining axes
for i in range(15, 18):
    axes[i].axis('off')

plt.suptitle('Sample Patches by Land Cover Type (False Color: NIR-R-G)', fontsize=14)
plt.tight_layout()
plt.show()

print("\nReal satellite data loaded successfully!")
print("Each patch is 64x64 pixels at 20m resolution (1.28km x 1.28km)")

### Demo: Feature Extraction


In [ ]:
# Helper function for feature extraction
def extract_features(model, images, device='cpu'):
    """Extract features from a model."""
    model.eval()
    images = images.to(device)
    with torch.no_grad():
        if hasattr(model, 'encode'):
            features = model.encode(images)
        else:
            features = model(images)
    return features

# Extract features from all patches
all_features = []
batch_size = 32

model.eval()
with torch.no_grad():
    for i in range(0, len(sample_patches), batch_size):
        batch = sample_patches[i:i+batch_size].to(device)
        features = extract_features(model, batch, device=str(device))
        all_features.append(features.cpu())

embeddings = torch.cat(all_features, dim=0).numpy()

print(f"Extracted embeddings shape: {embeddings.shape}")
print(f"Embedding dimension: {embeddings.shape[1]}")

### Demo: Visualize Embeddings


In [ ]:
# Helper function for visualization
def visualize_embeddings(embeddings, labels=None, method='pca', n_components=2):
    """Visualize high-dimensional embeddings in 2D."""
    from sklearn.decomposition import PCA
    from sklearn.manifold import TSNE

    if method == "pca":
        reducer = PCA(n_components=n_components)
    elif method == "tsne":
        reducer = TSNE(n_components=n_components, random_state=42)
    else:
        raise ValueError(f"Unknown method: {method}")

    reduced = reducer.fit_transform(embeddings)

    fig, ax = plt.subplots(figsize=(10, 8))

    if labels is not None:
        scatter = ax.scatter(
            reduced[:, 0], reduced[:, 1],
            c=labels, cmap='tab10', alpha=0.6, s=50
        )
        plt.colorbar(scatter, ax=ax, label='Class')
    else:
        ax.scatter(reduced[:, 0], reduced[:, 1], alpha=0.6, s=50)

    ax.set_xlabel(f'{method.upper()} Component 1')
    ax.set_ylabel(f'{method.upper()} Component 2')
    ax.set_title(f'Embedding Visualization ({method.upper()})')
    ax.grid(True, alpha=0.3)

    return fig

# Convert labels to numpy
labels_np = sample_labels.numpy()

# Visualize with PCA
fig = visualize_embeddings(
    embeddings,
    labels=labels_np,
    method='pca',
    n_components=2
)
plt.tight_layout()
plt.show()

print("Feature embeddings visualized using PCA")
print("Points with similar colors represent similar land cover types")

---

## Step 3: Simple Classification Task

Let's fine-tune a model on our sample data.

### Training Utilities → `geogfm/training/simple_trainer.py`


In [ ]:
"""Simple training utilities for classification tasks."""

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from typing import Dict, Optional, Tuple
import numpy as np


class SimpleDataset(Dataset):
    """Simple dataset for image patches and labels."""

    def __init__(self, images: torch.Tensor, labels: torch.Tensor):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]


def train_classifier(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: Optional[DataLoader] = None,
    n_epochs: int = 10,
    lr: float = 1e-4,
    device: str = "cpu"
) -> Dict:
    """
    Train a classifier.

    Args:
        model: Model to train
        train_loader: Training data loader
        val_loader: Validation data loader
        n_epochs: Number of epochs
        lr: Learning rate
        device: Device to use

    Returns:
        Training history
    """
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    for epoch in range(n_epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        # Validation
        if val_loader is not None:
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    val_total += labels.size(0)
                    val_correct += predicted.eq(labels).sum().item()

            val_loss /= len(val_loader)
            val_acc = val_correct / val_total

            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}: "
                  f"Train Loss={train_loss:.4f}, Train Acc={train_acc:.3f}")
            if val_loader is not None:
                print(f"  Val Loss={val_loss:.4f}, Val Acc={val_acc:.3f}")

    return history


def evaluate_model(
    model: nn.Module,
    test_loader: DataLoader,
    device: str = "cpu"
) -> Tuple[float, float]:
    """
    Evaluate a model.

    Args:
        model: Model to evaluate
        test_loader: Test data loader
        device: Device to use

    Returns:
        Test loss and accuracy
    """
    model.eval()
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()

    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    test_loss /= len(test_loader)
    test_acc = test_correct / test_total

    return test_loss, test_acc

### Demo: Train a Simple Classifier


In [ ]:
from sklearn.model_selection import train_test_split

# Helper classes and functions for training
class SimpleDataset(torch.utils.data.Dataset):
    """Simple dataset for image patches and labels."""
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

def train_classifier(model, train_loader, val_loader=None, n_epochs=10, lr=1e-4, device='cpu'):
    """Train a classifier."""
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    for epoch in range(n_epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        # Validation
        if val_loader is not None:
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    val_total += labels.size(0)
                    val_correct += predicted.eq(labels).sum().item()

            val_loss /= len(val_loader)
            val_acc = val_correct / val_total

            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.3f}")
            if val_loader is not None:
                print(f"  Val Loss={val_loss:.4f}, Val Acc={val_acc:.3f}")

    return history

# Split data
train_idx, val_idx = train_test_split(
    np.arange(len(sample_patches)),
    test_size=0.2,
    random_state=42,
    stratify=sample_labels.numpy()
)

train_images = sample_patches[train_idx]
train_labels = sample_labels[train_idx]
val_images = sample_patches[val_idx]
val_labels = sample_labels[val_idx]

# Create datasets
train_dataset = SimpleDataset(train_images, train_labels)
val_dataset = SimpleDataset(val_images, val_labels)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Training set: {len(train_dataset)} samples")
print(f"Validation set: {len(val_dataset)} samples")

# Create a simple classifier
class SimpleClassifier(nn.Module):
    def __init__(self, feature_extractor, num_classes=5):
        super().__init__()
        self.features = feature_extractor
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        features = self.features(x)
        return self.classifier(features)

classifier = SimpleClassifier(model, num_classes=5).to(device)

print(f"\nTraining classifier...")
history = train_classifier(
    classifier,
    train_loader,
    val_loader,
    n_epochs=15,
    lr=1e-3,
    device=str(device)
)

print("\nTraining complete!")

### Demo: Visualize Training Progress


In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

epochs = range(1, len(history['train_loss']) + 1)

ax1.plot(epochs, history['train_loss'], label='Train Loss')
ax1.plot(epochs, history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(epochs, history['train_acc'], label='Train Accuracy')
ax2.plot(epochs, history['val_acc'], label='Val Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final validation accuracy: {history['val_acc'][-1]:.3f}")

---

## Step 4: Model Comparison

Let's compare different approaches to understand trade-offs.

### Demo: Compare Feature Extraction vs. End-to-End Training


In [ ]:
# Compare frozen features vs. fine-tuned model

# Approach 1: Frozen feature extractor
frozen_model = SimpleClassifier(model, num_classes=5).to(device)
for param in frozen_model.features.parameters():
    param.requires_grad = False

print("Training with frozen features...")
frozen_history = train_classifier(
    frozen_model,
    train_loader,
    val_loader,
    n_epochs=10,
    lr=1e-3,
    device=str(device)
)

# Approach 2: End-to-end fine-tuning (we already have this from before)
print("\nComparing approaches:")
print(f"Frozen features - Final val acc: {frozen_history['val_acc'][-1]:.3f}")
print(f"Fine-tuned model - Final val acc: {history['val_acc'][-1]:.3f}")

# Visualize comparison
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(range(1, len(frozen_history['val_acc']) + 1),
        frozen_history['val_acc'],
        label='Frozen Features', marker='o')
ax.plot(range(1, len(history['val_acc']) + 1),
        history['val_acc'],
        label='Fine-tuned', marker='s')

ax.set_xlabel('Epoch')
ax.set_ylabel('Validation Accuracy')
ax.set_title('Comparison: Frozen vs. Fine-tuned Features')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Step 5: Real-World Application

Let's apply our trained model to make predictions on new data.

### Demo: Inference on New Patches


In [ ]:
# Extract additional patches from the scene for testing
# Use a different stride to get different patches
print("Extracting test patches...")
test_patches, _ = extract_patches_from_scene(
    best_scene,
    subset_bbox=santa_barbara_bbox,
    patch_size=64,
    stride=60,  # Different stride to get non-overlapping patches
    target_patches=50
)

test_labels = create_spectral_labels(test_patches)

print(f"Test set: {len(test_patches)} patches")

# Make predictions
classifier.eval()
all_preds = []
all_probs = []

with torch.no_grad():
    for i in range(0, len(test_patches), 32):
        batch = test_patches[i:i+32].to(device)
        outputs = classifier(batch)
        probs = torch.softmax(outputs, dim=1)
        preds = outputs.argmax(dim=1)

        all_preds.append(preds.cpu())
        all_probs.append(probs.cpu())

predictions = torch.cat(all_preds).numpy()
probabilities = torch.cat(all_probs).numpy()

# Calculate accuracy
accuracy = (predictions == test_labels.numpy()).mean()
print(f"\nTest accuracy: {accuracy:.3f}")

# Show confidence distribution
plt.figure(figsize=(10, 6))
max_probs = probabilities.max(axis=1)
plt.hist(max_probs, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Maximum Prediction Probability')
plt.ylabel('Frequency')
plt.title('Prediction Confidence Distribution')
plt.axvline(max_probs.mean(), color='red', linestyle='--',
            label=f'Mean: {max_probs.mean():.3f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Demo: Visualize Predictions


In [ ]:
# Visualize some predictions
class_names = ['Water', 'Vegetation', 'Urban', 'Bare Soil', 'Mixed']

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for idx in range(10):
    # Create false color composite (NIR, Red, Green) for better visualization
    # Channels: 0=Red, 1=Green, 2=Blue, 3=NIR
    nir = test_patches[idx, 3].numpy()
    red = test_patches[idx, 0].numpy()
    green = test_patches[idx, 1].numpy()

    # False color: NIR as red, red as green, green as blue
    img = np.stack([nir, red, green], axis=2)
    img = np.clip(img, 0, 1)

    true_label = test_labels[idx].item()
    pred_label = predictions[idx]
    confidence = probabilities[idx, pred_label]

    axes[idx].imshow(img)
    axes[idx].axis('off')

    color = 'green' if true_label == pred_label else 'red'
    title = f"True: {class_names[true_label]}\n"
    title += f"Pred: {class_names[pred_label]} ({confidence:.2f})"
    axes[idx].set_title(title, color=color, fontsize=9)

plt.suptitle('Sample Predictions - False Color (NIR-R-G)', fontsize=14)
plt.tight_layout()
plt.show()

print("\nNote: False color composite shown (NIR=Red, Red=Green, Green=Blue)")
print("Vegetation appears bright red due to high NIR reflectance")

---

## Conclusion

You've successfully worked with geospatial foundation models using TerraTorch!

### What You Accomplished

1. **Model Loading**: Loaded and inspected pretrained models
2. **Feature Extraction**: Extracted embeddings from satellite imagery
3. **Transfer Learning**: Fine-tuned models for classification
4. **Model Comparison**: Compared different training approaches
5. **Inference**: Applied models to make predictions on new data

### Key Takeaways

- **Pretrained models** provide powerful feature representations
- **Transfer learning** is more efficient than training from scratch
- **Feature extraction** vs. **fine-tuning** offer different trade-offs
- **Model comparison** helps identify the best approach for your task
- **TerraTorch** simplifies working with geospatial foundation models

### Performance Summary


In [ ]:
print("Training Summary")
print("=" * 50)
print(f"Model type: {type(classifier).__name__}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_patches)}")
print(f"\nFinal Results:")
print(f"Validation accuracy: {history['val_acc'][-1]:.3f}")
print(f"Test accuracy: {accuracy:.3f}")
print(f"Average confidence: {max_probs.mean():.3f}")

### Next Week Preview

In **Week 4**, we'll explore:

- Multi-modal foundation models (optical + radar)
- Temporal modeling with satellite time series
- Advanced fine-tuning strategies
- Scaling to larger datasets

Your understanding of foundation models provides the perfect foundation for these advanced topics!

## Resources

- [TerraTorch Documentation](https://github.com/IBM/terratorch)
- [Prithvi Model](https://huggingface.co/ibm-nasa-geospatial/Prithvi-100M)
- [Clay Foundation Model](https://clay-foundation.github.io/)
- [TorchGeo Documentation](https://torchgeo.readthedocs.io/)